* 给姚媛整理的疾病表格式  
* 根据姚媛的整理结果更新疾病库

In [1]:
import sys
sys.path.append('/home/ian/code/github')

In [2]:
from utils.pymongo_wrapper import pymongo_wrapper
pmw = pymongo_wrapper()
table = pmw.get_collection('jiankang39','diseases')

In [10]:
c=list(pmw.find_all(table,fieldlist=['疾病名称','简介','典型症状','多发人群']))

In [4]:
from utils.pandas_wrapper import pandas_wrapper as pdw

In [12]:
l1 = list()
l2 = list()
l3 = list()
for i in c:
    if '多发人群' in i:
        l1.append(i['多发人群'])
    else:
        l1.append(None)
    if '简介' in i:
        l2.append(i['简介'])
    else:
        l2.append(None)
    if '典型症状' in i:
        l3.append(','.join(i['典型症状']))
    else:
        l3.append(None)

In [17]:
df = pdw.build_df_from_dict({'疾病名称':[i['疾病名称'] for i in c],'简介':l2,'典型症状':l3,'多发人群':l1})
df.head()

NameError: name 'pdw' is not defined

In [15]:
df.to_csv('/home/ian/code/data/daozhen/39疾病列表.csv',encoding='utf8')

## 根据姚媛的整理结果更新疾病库

In [3]:
import pandas as pd

In [18]:
df = pd.read_excel('/home/ian/code/data/daozhen/39疾病列表-更新.xlsx').fillna('')
df.head()

,疾病名称,简介,典型症状,多发人群,罕见程度,多发性别1：男2：女,多发年龄,限定性别,限定年龄,备注,Unnamed: 10
0,偏头痛,偏头痛是一类有家族发病倾向的周期性发作疾病。表现为发作性的偏侧搏动性头痛，伴恶心、呕吐及羞明...,"单侧头痛,电击样痛或刺痛的头痛,头痛为搏动性",女性占2/3以上， 10岁前、20岁前、40岁前发病分别为25%， 55%， 90%。,2,2,,,,,
1,帕金森,帕金森病(Parkinsons disease)又称震颤麻痹，巴金森氏症或柏金逊症，多在60...,"静止性震颤,肌张力过高,感觉障碍",老年人，于40～70岁，60岁后发病率增高，30多岁前发病少见。,2,,67,,,,
2,老年痴呆症,阿尔茨海默病(Alzheimer’s disease，AD)是慢性进行性中枢神经系统变性病导...,,,1,,,,7,,
3,蛛网膜下腔出血,蛛网膜下腔出血(subarachnoid hemorrhage，SAH)是出血性脑血管病的一...,"脑膜刺激症状,定位神经体征,偏瘫",30～40岁青壮年多见，也有的报道80%的发病年龄在30～69岁。男性稍多于女性。,0,1,,,,,
4,小儿多动症,儿童多动症、多动综合征(hyperkinetic syndrome of childhood...,"注意力不集中,情绪起伏大,学习困难",婴幼儿,2,,,,23,,


In [21]:
df.shape

(7450, 11)

In [13]:
otherdict = {'普通':[0],'孕妇':[1],'产褥期':[2],'孕产妇':[1,2]}

In [20]:
for i,n in df['疾病名称'].items():
    try:
        d = table.find_one({'疾病名称':n})
        d['罕见程度'] = int(df.iloc[i,4])
        if df.iloc[i,5]:
            d['多发性别'] = [int(df.iloc[i,5])-1]
        else:
            d['多发性别'] = [0,1]

        if df.iloc[i,6]:
            d['多发年龄'] = list(str(int(df.iloc[i,6])))
        else:
            d['多发年龄'] = ['1', '2', '3', '4', '5', '6', '7']
        if df.iloc[i,7]:
            d['限定性别'] = [int(df.iloc[i,7])-1]
        else:
            d['限定性别'] = [0,1]
        if df.iloc[i,8]:
            d['限定年龄'] = list(str(int(df.iloc[i,8])))
        else:
            d['限定年龄'] = ['1', '2', '3', '4', '5', '6', '7']
        if df.iloc[i,9]:
            if df.iloc[i,9] in otherdict:
                d['备注'] = otherdict[df.iloc[i,9]]
        else:
            d['备注'] = [0,1,2]
        pmw.update_doc(table,{'_id':d['_id']},d)
    except Exception as e:
        print(e)
        print(i,n)

In [9]:
type(df.iloc[0,5])

float

In [12]:
list('1234567')

['1', '2', '3', '4', '5', '6', '7']